In [ ]:
import numpy as np
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
tqdm.pandas()

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().tolist()
    embedding_str = ' '.join(map(str, embedding))
    return embedding_str

In [ ]:
with open('hints_synthlora_qwen32b_140steps.pkl', 'rb') as f: # тут укажите pkl, который был получен после инференса модели
    hints = pickle.load(f)

In [ ]:
len([x for x in hints if len(x) < 2])

In [ ]:
sample = pd.read_csv('/kaggle/input/hse-code-hack/data_v0/for_teams/submit_example.csv') # тут укажите путь к submit_example
sample['author_comment'] = hints

In [ ]:
sample['author_comment_embedding'] = sample['author_comment'].progress_apply(get_sentence_embedding)

In [ ]:
sample.iloc[156].author_comment

In [ ]:
sample.to_csv('hints_synthlora_qwen32b_140steps.csv', index=False) # получите итоговый сабмит